## Pre-trained models for automatic speech recognition

In [ ]:
!pip install datasets
!pip install soundfile
!pip install librosa

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install -U huggingface_hub

### Probing CTC Models

In [ ]:
from datasets import load_dataset
dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
dataset

In [ ]:
import IPython
sample = dataset[2]
print(sample["text"])
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-100h")

In [ ]:
pipe(sample["audio"].copy())

### Graduation to Seq2Seq

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

In [ ]:
pipe(sample["audio"], max_new_tokens=256)

#### Multilingual

In [ ]:
dataset = load_dataset("facebook/multilingual_librispeech", "dutch", split="validation", streaming=True)
sample = next(iter(dataset))

In [ ]:
print(sample["text"])
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])


In [ ]:
pipe(sample["audio"].copy(), max_new_tokens=256, generate_kwargs={"task":"transcribe"})

In [ ]:
pipe(sample["audio"].copy(), max_new_tokens=256, generate_kwargs={"task":"translate"})

In [ ]:
dataset = load_dataset("facebook/multilingual_librispeech", "spanish", split="validation", streaming=True)
sample = next(iter(dataset))

In [ ]:
print(sample["text"])
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])


In [ ]:
pipe(sample["audio"].copy(), max_new_tokens=256, generate_kwargs={"task":"transcribe"})

In [ ]:
pipe(sample["audio"].copy(), max_new_tokens=256, generate_kwargs={"task":"translate"})

### Long-Form Transcription and Timestamps

In [ ]:
import numpy as np
target_length_in_m = 5
# convert from minutes to seconds(* 60) to num samples(* sampling_rate)
sampling_rate = pipe.feature_extractor.sampling_rate
target_length_in_samples = target_length_in_m * 60 * sampling_rate
long_audio = []
for sample in dataset:
    long_audio.extend(sample["audio"]["array"])
    if len(long_audio) > target_length_in_samples:
        break
        
long_audio = np.asarray(long_audio)

# how did we do

seconds = len(long_audio) / 16_000
minutes, seconds = divmod(seconds, 60)

print(f"Length of audio sample is {minutes} minutes {seconds:.2f} seconds")


In [ ]:
pipe (
    long_audio,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30, # 30 second chunks
    batch_size=8 # batch of 8 chunks at a time
)

In [ ]:
pipe (
    long_audio,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30, # 30 second chunks
    batch_size=8, # batch of 8 chunks at a time
    return_timestamps=True #return timestamps for annotating video
)

## Evaluation and metrics for speech recognition

In [ ]:
reference = "the cat sat on the mat"
prediction = "the cat sit on the"

### Word Error Rate

In [ ]:
!pip install --upgrade evaluate jiwer

In [ ]:
from evaluate import load
wer_metric = load("wer")
wer = wer_metric.compute(references=[reference], predictions=[prediction])
wer

### Word accuracy

W Acc = 1 - WER

### Character error rate

In [ ]:
cer_metric = load("cer")
cer = cer_metric.compute(references=[reference], predictions=[prediction])
cer

### Normalisation

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
normalizer = BasicTextNormalizer()
prediction = " He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind."
normalized_prediction = normalizer(prediction)
normalized_prediction

In [ ]:
reference = "HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND"
normalized_reference = normalizer(reference)

wer = wer_metric.compute(
    references=[normalized_reference], predictions=[normalized_prediction]
)
wer

### Putting it all together

In [ ]:
# have to login
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import pipeline
import torch

if torch.cuda.is_available():
    device="cuda:0"
    torch_dtype = torch.float16
else:
    device="cpu"
    torch_dtype=torch.float32
    
pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    torch_dtype=torch_dtype,
    device=device,
)


In [ ]:
from datasets import load_dataset
common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_13_0", "dv", split="test", use_auth_token=True)

In [ ]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

all_predictions =[]

# rucommon_voice_testreamed inference
for prediction in tqdm(
    pipe(
        KeyDataset(common_voice_test, "audio"),
        max_new_tokens=128,
        generate_kwargs={"task": "transcribe"},
        batch_size=32,
    ),
    total=len(common_voice_test),
):
    all_predictions.append(prediction["text"])

In [ ]:
from evaluate import load
wer_metric = load("wer")
wer_ortho = 100* wer_metric.compute(
    references=common_voice_test["sentence"], predictions=all_predictions
)
wer_ortho

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
normalizer = BasicTextNormalizer()

In [ ]:
# compute normalised WER
all_predictions_norm = [normalizer(pred) for pred in all_predictions]
all_references_norm = [normalizer(label) for label in common_voice_test["sentence"]]

In [ ]:
# filtering step to only evaluate the samples that correspond to non-zero-references
all_predictions_norm = [
    all_predictions_norm[i] for i in range(len(all_predictions_norm)) if len(all_references_norm[i]) > 0
]
all_references_norm = [
    all_references_norm[i] for i in range(len(all_references_norm)) if len(all_references_norm[i]) > 0
]
wer = 100 * wer_metric.compute(
    references=all_references_norm, predictions=all_predictions_norm
)

wer
                                                                         

## Fine-tune ASR with Trainer API

### Prepare Environment

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Load Dataset

In [1]:
from datasets import load_dataset, DatasetDict
common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_13_0", "dv", split="train+validation")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_13_0", "dv", split="test")

common_voice

Found cached dataset common_voice_13_0 (/home/studio-lab-user/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/dv/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055)
Found cached dataset common_voice_13_0 (/home/studio-lab-user/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/dv/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4904
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2212
    })
})

In [2]:
common_voice = common_voice.select_columns(["audio", "sentence"])
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4904
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2212
    })
})

### Feature Extractor, Tokenizer and Processor